In [3]:
import pandas as pd

In [4]:
data = pd.read_csv(r"reddit_python.csv")
df = data[['upvote_ratio', 'id', 'author', 'num_comments', 'title', 'selftext']]
df['selftext'] = df['selftext'].fillna(df['title'])
df.selftext = df.selftext.str.lower()

<ipython-input-4-b3cfbbaf98d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['selftext'] = df['selftext'].fillna(df['title'])
<ipython-input-4-b3cfbbaf98d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.selftext = df.selftext.str.lower()


In [5]:
df.head()

,upvote_ratio,id,author,num_comments,title,selftext
0,0.94,1aq1liy,empyreandreams,0,/r/Exercise Beginners Guide &amp; Instructiona...,/r/exercise beginners guide &amp; instructiona...
1,0.97,1czmuou,empyreandreams,8,Dancing is a great way to exercise. THIS is th...,dancing is a great way to exercise. this is th...
2,1.00,1f0dnyc,Life_Chemist9642,0,Sprints,so my current cardio routine is 3 days a week ...
3,1.00,1f0d6t9,BeVanderhill,0,R/exercise,what upper body strength exercises can i do wh...
4,1.00,1f0epqi,hcconn,2,Building muscle after being bedbound?,"34f 170lbs and 5'5""\n\ni had a pretty major su..."


In [6]:
pip install transformers -q

In [7]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
def get_emotion(text):
  label = pipe(text)
  return label[0]['label']

In [9]:
df['emotion'] = df['selftext'][:10].apply(get_emotion)

<ipython-input-9-bcaeac955497>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emotion'] = df['selftext'][:10].apply(get_emotion)


In [10]:
cols = df.loc[:10, ['selftext', 'emotion']]
cols

,selftext,emotion
0,/r/exercise beginners guide &amp; instructiona...,neutral
1,dancing is a great way to exercise. this is th...,admiration
2,so my current cardio routine is 3 days a week ...,curiosity
3,what upper body strength exercises can i do wh...,curiosity
4,"34f 170lbs and 5'5""\n\ni had a pretty major su...",curiosity
5,"hey all.\n\ni am in a relationship ( married, ...",love
6,"i mean like butt touching heel. if not, what o...",curiosity
7,why the same exercise leads to different fat l...,neutral
8,random quick question. how do so many people n...,disgust
9,"hi guys, i am completely new to this, and yes ...",desire


In [11]:
df.shape

(502, 7)

In [12]:
df['emotion'] = df['selftext'][:10].apply(get_emotion)

<ipython-input-12-bcaeac955497>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emotion'] = df['selftext'][:10].apply(get_emotion)


In [ ]:
def split_text_into_chunks(text, max_length=512):
    """
    Splits the input text into smaller chunks of max_length tokens.
    """
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

def detect_emotions_in_batches_with_chunks(df, text_column, batch_size=10, max_length=512):
    emotions = []
    for i in range(0, len(df), batch_size):
        batch_texts = df[text_column].iloc[i:i+batch_size].tolist()

        batch_emotions = []
        for text in batch_texts:
            # Split text into chunks
            chunks = split_text_into_chunks(text, max_length=max_length)

            # Detect emotion for each chunk
            chunk_emotions = pipe(chunks)

            # Aggregate the emotions (you can choose different strategies here)
            # For now, we take the most frequent emotion among the chunks
            chunk_emotion_labels = [result['label'] for result in chunk_emotions]
            most_frequent_emotion = max(set(chunk_emotion_labels), key=chunk_emotion_labels.count)
            batch_emotions.append(most_frequent_emotion)

        emotions.extend(batch_emotions)

    df['emotion'] = emotions[:len(df)]


detect_emotions_in_batches_with_chunks(df, text_column='selftext')

print(df.head())


In [ ]:
df.to_csv('processed_emotions2.csv', index=False)

In [ ]:
data1 = pd.read_csv('/content/reddit_python1.csv')
df1 = data1[['upvote_ratio', 'id', 'author', 'num_comments', 'title', 'selftext']]
df1['selftext'] = df1['selftext'].fillna(df1['title'])
df1.selftext = df1.selftext.str.lower()

In [ ]:
detect_emotions_in_batches_with_chunks(df1, text_column='selftext')
df1.to_csv('processed_emotions_d.csv', index=False)